The dataset can be downloaded from here and it consists of 18000 grayscale images (18000x150x150
or 18000x75x75) contained in ‘images.npy’. The labels for each sample are represented by two integers
(18000x2, ‘labels.npy’ file), that correspond to the hour and minute displayed by the clock. You can see
that each image is rendered from a different angle and rotation and they might contain light reflections from
within the scene making this a non-trivial problem. For your experiments, we suggest splitting your data
into 80/10/10% splits for training/validation and test sets respectively. Remember to shuffle your dataset
as the sample files are ordered. We suggest using the smaller dataset for your initial tests and runs (75x75
images) and then reporting your results on the larger (150x150) datase

## GPU CHECK

In [214]:
import tensorflow  as tf
print(tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"TensorFlow detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu.name}")
else:
    print("TensorFlow did NOT detect any GPUs. It will use the CPU.")


2.20.0
TensorFlow detected 1 GPU(s):
  - /physical_device:GPU:0


#### import needed packages

In [2]:

# from tensorflow import keras
import os
# import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras


#### load data

In [3]:
data_folder = "A1_data_75"
images_path = os.path.join(data_folder, "images.npy")
images = np.load(images_path)
labels_path = os.path.join(data_folder, "labels.npy")
labels = np.load(labels_path)


(a) Classification - treat this as a n-class classification problem. We suggest starting out with a
smaller number of categories e.g. grouping all the samples that are between [3 : 00 −3 : 30] into
a single category (results in 24 categories in total), and trying to train a CNN model. Once you
have found a working architecture, increase the number of categories by using smaller intervals
for grouping samples to increase the ’common sense accuracy’. Can you train a network using
all 720 different labels? What problems does such a label representation have?

## Task a: classification
We will start with deviding labels into 24 categories, one for each 30 minute

In [ ]:

print(labels)
def get_cat_labels(labels):
    new_labels = []
    for label in labels:
        label = label[0]* 2 + int(label[1] >= 30)
        new_labels.append(label)
    return np.array(new_labels)
labels = get_cat_labels(labels)
print(labels)


[[ 0  0]
 [ 0  0]
 [ 0  0]
 ...
 [11 59]
 [11 59]
 [11 59]]
[ 0  0  0 ... 23 23 23]


We then split the data into training, validation, and test sets. The sklearn train_test_split method shuffles the data by default

In [ ]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1.


X_train shape: (14400, 75, 75)
y_train shape: (14400,)
X_valid shape: (1800, 75, 75)
y_valid shape: (1800,)
X_test shape: (1800, 75, 75)
y_test shape: (1800,)


we define a common sense loss. This will calculate how far of the prediction was

In [9]:
def common_sense_loss(y_true, y_pred):
    """
    """
    y_pred_class = tf.argmax(y_pred, axis=1)
    y_true_float = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
    y_pred_float = tf.cast(y_pred_class, dtype=tf.float32)
    diff = tf.abs(y_true_float - y_pred_float)
    cyclical_diff = tf.minimum(diff, 12.0 - diff)
    print(cyclical_diff)
    return tf.reduce_mean(cyclical_diff)


Our model for 24 class classification. We use a scheduler to lower the learning rate when we plateau



In [ ]:

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,          # halve the learning rate if there is no improvement
    patience=3,          # Wait 2 epochs with no improvement before reducing
    min_lr=1e-6          # Set a minimum learning rate at 1e-6
)
early_stopper = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,          # Wait 6 epochs for improvement before stopping
    restore_best_weights=True  # Automatically restore the weights from the best epoch
)
model = keras.models.Sequential([
    keras.Input(shape=(75, 75, 1)),
    # Block 1
    keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    # Block 2
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2),

    # Block 3
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (9, 9, 128)

    # Block 4
    keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (4, 4, 256)

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(24, activation="softmax")
])
model.compile(loss='sparse_categorical_crossentropy',
optimizer=keras.optimizers.Adam(learning_rate=0.001),
metrics=[common_sense_loss,"Accuracy"
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ],
)


In [ ]:
model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper]
    )
#evaluate the model on the test set
test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#base:0.8420000076293945
#leaky: 0.8525000214576721
#leaky + L2regularization: 0.8472999930381775
#leaky + batch normalization: 0.8978000283241272

(print(tf.__version__))
#0.9711111187934875
# metrics=["accuracy"])

Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - Accuracy: 0.9581 - common_sense_loss: 0.0559 - loss: 0.1244 - val_Accuracy: 0.9578 - val_common_sense_loss: 0.0351 - val_loss: 0.1369 - learning_rate: 3.1250e-05
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - Accuracy: 0.9596 - common_sense_loss: 0.0474 - loss: 0.1199 - val_Accuracy: 0.9594 - val_common_sense_loss: 0.0389 - val_loss: 0.1259 - learning_rate: 3.1250e-05
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - Accuracy: 0.9624 - common_sense_loss: 0.0453 - loss: 0.1182 - val_Accuracy: 0.9517 - val_common_sense_loss: 0.0515 - val_loss: 0.1607 - learning_rate: 3.1250e-05
Epoch 4/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - Accuracy: 0.9633 - common_sense_loss: 0.0463 - loss: 0.1117 - val_Accuracy: 0.9589 - val_common_sense_loss: 0.0417 - val_loss: 0.1299 - learning_rate: 3.1250e-05
Epoch 5/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - Accuracy: 0.9642 - common_sense_loss: 0.0422 - loss: 0.1092 - val_Accuracy: 0

We now make a class for every 10 minutes

In [ ]:
labels = np.load(labels_path)

def get_cat_labels_10(labels):
    new_labels = []
    dct = {}
    for label in labels:
        old = label
        label = label[0]* 6 + int((label[1])/10)
        new_labels.append(label)
        dct[str(old)] = label
    print(dct)
    return np.array(new_labels)
labels = get_cat_labels_10(labels)


{'[0 0]': np.int64(0), '[0 1]': np.int64(0), '[0 2]': np.int64(0), '[0 3]': np.int64(0), '[0 4]': np.int64(0), '[0 5]': np.int64(0), '[0 6]': np.int64(0), '[0 7]': np.int64(0), '[0 8]': np.int64(0), '[0 9]': np.int64(0), '[ 0 10]': np.int64(1), '[ 0 11]': np.int64(1), '[ 0 12]': np.int64(1), '[ 0 13]': np.int64(1), '[ 0 14]': np.int64(1), '[ 0 15]': np.int64(1), '[ 0 16]': np.int64(1), '[ 0 17]': np.int64(1), '[ 0 18]': np.int64(1), '[ 0 19]': np.int64(1), '[ 0 20]': np.int64(2), '[ 0 21]': np.int64(2), '[ 0 22]': np.int64(2), '[ 0 23]': np.int64(2), '[ 0 24]': np.int64(2), '[ 0 25]': np.int64(2), '[ 0 26]': np.int64(2), '[ 0 27]': np.int64(2), '[ 0 28]': np.int64(2), '[ 0 29]': np.int64(2), '[ 0 30]': np.int64(3), '[ 0 31]': np.int64(3), '[ 0 32]': np.int64(3), '[ 0 33]': np.int64(3), '[ 0 34]': np.int64(3), '[ 0 35]': np.int64(3), '[ 0 36]': np.int64(3), '[ 0 37]': np.int64(3), '[ 0 38]': np.int64(3), '[ 0 39]': np.int64(3), '[ 0 40]': np.int64(4), '[ 0 41]': np.int64(4), '[ 0 42]': 

In [42]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default


In [ ]:
def common_sense_loss(y_true, y_pred):
    """
    """
    y_pred_class = tf.argmax(y_pred, axis=1)
    y_true_float = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
    y_pred_float = tf.cast(y_pred_class, dtype=tf.float32)
    diff = tf.abs(y_true_float - y_pred_float)
    cyclical_diff = tf.minimum(diff, 12.0 - diff)
    print(cyclical_diff)
    return tf.reduce_mean(cyclical_diff)

In [47]:
max_pool = keras.layers.MaxPool2D(pool_size=2)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,          # halce the learning rate if no improvement
    patience=2,          # Wait 2 epochs with no improvement before reducing
    min_lr=1e-6          # Set a minimum learning rate at 1e-6
)
early_stopper = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,          # Wait 5 epochs for improvement before stopping
    restore_best_weights=True  # Automatically restore the model weights from the best epoch
)
# avg_pool = keras.layers.AveragePooling2D(pool_size=2)
model = keras.models.Sequential([
    keras.Input(shape=(75, 75, 1)),
    # Block 1
    keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)), # Output shape: (37, 37, 32)

    # Block 2
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (18, 18, 64)

    # Block 3
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (9, 9, 128)

    # Block 4
    keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (4, 4, 256)

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(72, activation="softmax")
])
model.compile(loss='sparse_categorical_crossentropy',
optimizer=keras.optimizers.Adam(learning_rate=0.001),
metrics=[common_sense_loss,"Accuracy"
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ],
)


In [ ]:
model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper]
    )
#evaluate the model on the test set
test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#base:0.8420000076293945
#leaky: 0.8525000214576721
#leaky + L2regularization: 0.8472999930381775
#leaky + batch normalization: 0.8978000283241272

(print(tf.__version__))
#0.9194444417953491
# metrics=["accuracy"])

Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 0.9309 - common_sense_loss: 0.0900 - loss: 0.2031 - val_Accuracy: 0.9156 - val_common_sense_loss: 0.1091 - val_loss: 0.2537 - learning_rate: 3.1250e-05
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 0.9324 - common_sense_loss: 0.0492 - loss: 0.1923 - val_Accuracy: 0.9122 - val_common_sense_loss: 0.0735 - val_loss: 0.2550 - learning_rate: 3.1250e-05
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - Accuracy: 0.9317 - common_sense_loss: 0.0530 - loss: 0.1989 - val_Accuracy: 0.9100 - val_common_sense_loss: 0.0707 - val_loss: 0.2669 - learning_rate: 3.1250e-05
Epoch 4/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - Accuracy: 0.9408 - common_sense_loss: 0.0096 - loss: 0.1797 - val_Accuracy: 0.9133 - val_common_sense_loss: 0.0768 - val_loss: 0.2498 - learning_rate: 1.5625e-05
Epoch 5/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - Accuracy: 0.9401 - common_sense_loss: 0.0856 - loss: 0.1767 - val_Accuracy: 0.9122

In [84]:
labels = np.load(labels_path)
def get_cat_labels_10(labels):
    new_labels = []
    dct = {}
    for label in labels:
        old = label
        label = label[0]* 60 + int((label[1]))
        new_labels.append(label)
        dct[str(old)] = label
    print(dct)
    return np.array(new_labels)
labels = get_cat_labels_10(labels)

{'[0 0]': np.int64(0), '[0 1]': np.int64(1), '[0 2]': np.int64(2), '[0 3]': np.int64(3), '[0 4]': np.int64(4), '[0 5]': np.int64(5), '[0 6]': np.int64(6), '[0 7]': np.int64(7), '[0 8]': np.int64(8), '[0 9]': np.int64(9), '[ 0 10]': np.int64(10), '[ 0 11]': np.int64(11), '[ 0 12]': np.int64(12), '[ 0 13]': np.int64(13), '[ 0 14]': np.int64(14), '[ 0 15]': np.int64(15), '[ 0 16]': np.int64(16), '[ 0 17]': np.int64(17), '[ 0 18]': np.int64(18), '[ 0 19]': np.int64(19), '[ 0 20]': np.int64(20), '[ 0 21]': np.int64(21), '[ 0 22]': np.int64(22), '[ 0 23]': np.int64(23), '[ 0 24]': np.int64(24), '[ 0 25]': np.int64(25), '[ 0 26]': np.int64(26), '[ 0 27]': np.int64(27), '[ 0 28]': np.int64(28), '[ 0 29]': np.int64(29), '[ 0 30]': np.int64(30), '[ 0 31]': np.int64(31), '[ 0 32]': np.int64(32), '[ 0 33]': np.int64(33), '[ 0 34]': np.int64(34), '[ 0 35]': np.int64(35), '[ 0 36]': np.int64(36), '[ 0 37]': np.int64(37), '[ 0 38]': np.int64(38), '[ 0 39]': np.int64(39), '[ 0 40]': np.int64(40), '[ 0

In [87]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default
print(y_train)


[ 57 587 472 ... 223 268 344]


In [ ]:
def common_sense_loss(y_true, y_pred):
    """
    """
    y_pred_class = tf.argmax(y_pred, axis=1)
    y_true_float = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
    y_pred_float = tf.cast(y_pred_class, dtype=tf.float32)
    diff = tf.abs(y_true_float - y_pred_float)
    cyclical_diff = tf.minimum(diff, 720.0 - diff)
    print(cyclical_diff)
    return tf.reduce_mean(cyclical_diff)


In [172]:
# max_pool = keras.layers.MaxPool2D(pool_size=2)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,          # halce the learning rate if no improvement
    patience=5,          # Wait 4 epochs with no improvement before reducing
    min_lr=1e-9         # Set a minimum learning rate at 1e-6
)
early_stopper = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,          # Wait 8 epochs for improvement before stopping
    restore_best_weights=True  # Automatically restore the model weights from the best epoch
)
# avg_pool = keras.layers.AveragePooling2D(pool_size=2)
model = keras.models.Sequential([
    keras.Input(shape=(75, 75, 1)),
    # Block 1
    keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)), # Output shape: (37, 37, 32)
    keras.layers.Dropout(0.15),
    # Block 2
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (18, 18, 64)
    keras.layers.Dropout(0.15),
    # Block 3
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (9, 9, 128)
    keras.layers.Dropout(0.15),
    # Block 4
    keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (4, 4, 256)

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(720, activation="softmax")
])
model.compile(loss='sparse_categorical_crossentropy',
optimizer=keras.optimizers.SGD(learning_rate=0.0001),
metrics=["Accuracy", common_sense_loss
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ],
)


In [140]:
model.fit(
    X_train, y_train,
    epochs=60,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper]
    )
#evaluate the model on the test set
test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)
(print(tf.__version__))
#0.9194444417953491
# metrics=["accuracy"])

Epoch 1/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - Accuracy: 0.0018 - common_sense_loss: 180.7997 - loss: 7.6930 - val_Accuracy: 0.0017 - val_common_sense_loss: 181.3591 - val_loss: 6.7426 - learning_rate: 1.0000e-04
Epoch 2/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 9.0278e-04 - common_sense_loss: 180.3764 - loss: 7.2143 - val_Accuracy: 0.0000e+00 - val_common_sense_loss: 181.4065 - val_loss: 6.7131 - learning_rate: 1.0000e-04
Epoch 3/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 0.0012 - common_sense_loss: 180.1556 - loss: 7.0075 - val_Accuracy: 0.0011 - val_common_sense_loss: 181.4947 - val_loss: 6.6875 - learning_rate: 1.0000e-04
Epoch 4/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 0.0019 - common_sense_loss: 180.2813 - loss: 6.9012 - val_Accuracy: 0.0033 - val_common_sense_loss: 181.5918 - val_loss: 6.7030 - learning_rate: 1.0000e-04
Epoch 5/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - Accuracy: 0.0017 - common_sense_loss: 180.1073 - loss: 6.

In [ ]:
#feature prep for dual head

labels = np.load(labels_path)


(18000, 2)


In [ ]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default

[[ 0 57]
 [ 9 47]
 [ 7 52]
 ...
 [ 3 43]
 [ 4 28]
 [ 5 44]]


In [206]:
y_train_hours = y_train[:, 0]
y_train_minutes = y_train[:, 1]
y_valid_hours = y_valid[:, 0]
y_valid_minutes = y_valid[:, 1]
y_test_hours = y_test[:, 0]
y_test_minutes = y_test[:, 1]

#one-hot encoding the labels
y_train_hours_enc = keras.utils.to_categorical(y_train_hours, num_classes=12)
y_train_minutes_enc = y_train_minutes
y_valid_hours_enc = keras.utils.to_categorical(y_valid_hours, num_classes=12)
y_valid_minutes_enc = y_valid_minutes
y_test_hours_enc = keras.utils.to_categorical(y_test_hours, num_classes=12)
y_test_minutes_enc = y_test_minutes
y_train_formatted = {
    "hour_output": y_train_hours_enc,
    "minute_output": y_train_minutes_enc
}

y_valid_formatted = {
    "hour_output": y_valid_hours_enc,
    "minute_output": y_valid_minutes_enc
}
y_train_formatted = {
    "hour_output": y_train_hours_enc,
    "minute_output": y_train_minutes_enc
}

y_valid_formatted = {
    "hour_output": y_valid_hours_enc,
    "minute_output": y_valid_minutes_enc
}


In [211]:
inputs = keras.Input(shape=(75, 75, 1), name="input_image")
x = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D((2,2))(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(256, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation="leaky_relu")(x)
shared_features = keras.layers.Dropout(0.5)(x)
hour_branch = keras.layers.Dense(64, activation="leaky_relu")(shared_features)
hour_branch = keras.layers.Dropout(0.5)(hour_branch)
hour_output = keras.layers.Dense(12, activation="softmax", name="hour_output")(hour_branch)
minute_branch = keras.layers.Dense(128, activation="leaky_relu")(shared_features)
minute_branch = keras.layers.Dropout(0.5)(minute_branch)
minute_output = keras.layers.Dense(1, activation="linear", name="minute_output")(minute_branch)

model = keras.Model(inputs=inputs, outputs=[hour_output, minute_output])
minute_loss_weight = 0.005 #mse is a lot higher than crossentropy.
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={
        "hour_output": "categorical_crossentropy",
        "minute_output": "mean_squared_error"
    },
    loss_weights={
        "hour_output": 1,
        "minute_output": minute_loss_weight
    },
    metrics={
        "hour_output": "accuracy",
        "minute_output": "mean_absolute_error",
    }
)
model.summary()

Model: "functional_68"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 75, 75, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_414 (Conv2D) │ (None, 75, 75,    │        320 │ input_image[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        128 │ conv2d_414[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_315   │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_282         │ (None, 37, 37,    │          0 │ max_pooling2d_31… │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_415 (Conv2D) │ (None, 37, 37,    │     18,496 │ dropout_282[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        256 │ conv2d_415[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_416 (Conv2D) │ (None, 37, 37,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        256 │ conv2d_416[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_316   │ (None, 18, 18,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_283         │ (None, 18, 18,    │          0 │ max_pooling2d_31… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_417 (Conv2D) │ (None, 18, 18,    │     73,856 │ dropout_283[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18, 18,    │        512 │ conv2d_417[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_418 (Conv2D) │ (None, 18, 18,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18, 18,    │        512 │ conv2d_418[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_317   │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_284         │ (None, 9, 9, 128) │          0 │ max_pooling2d_31

 Total params: 2,772,109 (10.57 MB)

 Trainable params: 2,770,765 (10.57 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [221]:
import keras.backend as K

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.10,          # halce the learning rate if no improvement
    patience=4,          # Wait 4 epochs with no improvement before reducing
    min_lr=1e-9         # Set a minimum learning rate at 1e-6
)
early_stopper = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,          # Wait 8 epochs for improvement before stopping
    restore_best_weights=True  # Automatically restore the model weights from the best epoch
)
model.fit(
    X_train, y_train_formatted,
    epochs=60,
    validation_data=(X_valid, y_valid_formatted),
    callbacks=[lr_scheduler, early_stopper]
    )
# K.clear_session()

Epoch 1/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - hour_output_accuracy: 0.8589 - hour_output_loss: 0.3679 - loss: 0.6612 - minute_output_loss: 58.6716 - minute_output_mean_absolute_error: 5.6138 - val_hour_output_accuracy: 0.9183 - val_hour_output_loss: 0.2210 - val_loss: 0.4366 - val_minute_output_loss: 42.3026 - val_minute_output_mean_absolute_error: 4.2366 - learning_rate: 1.0000e-04
Epoch 2/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - hour_output_accuracy: 0.8716 - hour_output_loss: 0.3297 - loss: 0.6128 - minute_output_loss: 56.6232 - minute_output_mean_absolute_error: 5.5364 - val_hour_output_accuracy: 0.8894 - val_hour_output_loss: 0.2821 - val_loss: 0.5237 - val_minute_output_loss: 47.5338 - val_minute_output_mean_absolute_error: 4.8370 - learning_rate: 1.0000e-04
Epoch 3/60
450/450 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - hour_output_accuracy: 0.8827 - hour_output_loss: 0.3124 - loss: 0.5894 - minute_output_loss: 55.4113 - minute_output_mean_absolute_error: 5.4749 - val_hour

In [246]:
def common_sense_loss(y_true, y_pred):

        y_true_hour = np.argmax(y_true["hour_output"], axis=1)
        y_true_minute = np.array(y_true["minute_output"], dtype=float)
        y_pred_hour = np.argmax(y_pred[0], axis=1)
        y_pred_minute = y_pred[1].squeeze()
        y_true_total_minutes = y_true_hour * 60 + y_true_minute
        y_pred_total_minutes = y_pred_hour * 60 + y_pred_minute
        diff = y_true_total_minutes - y_pred_total_minutes
        abs_diff = tf.abs(diff)


        cyclical_error = tf.minimum(abs_diff, 720.0 - abs_diff)

        return cyclical_error

In [ ]:
y_test_minutes_enc = np.array(y_test_minutes, dtype=float)
model.evaluate(X_test, {
    "hour_output": y_test_hours_enc,
    "minute_output": y_test_minutes_enc
})
y_test_formatted = {
    "hour_output": y_test_hours_enc,
    "minute_output": y_test_minutes_enc
}
y_pred_list = model.predict(X_test)

np.mean((common_sense_loss(y_test_formatted, y_pred_list)))


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - hour_output_accuracy: 0.9711 - hour_output_loss: 0.0992 - loss: 0.2942 - minute_output_loss: 38.4920 - minute_output_mean_absolute_error: 4.0071
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[array([[6.6411525e-07, 1.4670831e-08, 7.9728916e-06, ..., 7.8694655e-12,
        7.0172508e-09, 1.5302958e-06],
       [3.2581678e-16, 1.1008986e-18, 1.2715607e-20, ..., 6.3151143e-02,
        9.3684882e-01, 1.4551491e-08],
       [8.7864156e-04, 9.9865305e-01, 4.6823893e-04, ..., 5.0802844e-18,
        1.8039712e-13, 5.5778993e-10],
       ...,
       [1.2183864e-11, 1.2733003e-03, 9.9713576e-01, ..., 6.1500063e-21,
        1.4436660e-19, 2.2348427e-19],
       [6.2379233e-15, 7.2343755e-16, 5.9267478e-14, ..., 2.8021548e-15,
        4.6440737e-15, 3.8109283e-14],
       [9.4759143e-16, 1.2355712e-16, 3.0967093e-11, ..., 4.2677498e-20,
        4.2906731e-18, 3.1026256e-16]], shape=(1800, 12), dtype=float32), array([[26.36074  ],
       [ 7.9434767],
       [51.61

np.float64(5.15378174846371)

Model: "functional_52"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 75, 75, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_318 (Conv2D) │ (None, 75, 75,    │        320 │ input_image[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        128 │ conv2d_318[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_251   │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_186         │ (None, 37, 37,    │          0 │ max_pooling2d_25… │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_319 (Conv2D) │ (None, 37, 37,    │     18,496 │ dropout_186[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        256 │ conv2d_319[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_320 (Conv2D) │ (None, 37, 37,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        256 │ conv2d_320[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_252   │ (None, 18, 18,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_187         │ (None, 18, 18,    │          0 │ max_pooling2d_25… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_321 (Conv2D) │ (None, 18, 18,    │     73,856 │ dropout_187[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18, 18,    │        512 │ conv2d_321[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_322 (Conv2D) │ (None, 18, 18,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18, 18,    │        512 │ conv2d_322[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_253   │ (None, 9, 9, 128) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_188         │ (None, 9, 9, 128) │          0 │ max_pooling2d_25

 Total params: 2,779,720 (10.60 MB)

 Trainable params: 2,778,376 (10.60 MB)

 Non-trainable params: 1,344 (5.25 KB)